In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartForConditionalGeneration, AutoTokenizer,AutoModelForSeq2SeqLM,BertTokenizerFast
import torch
from transformers import pipeline
import pandas as pd 
import numpy as np
import pyarrow.parquet as pq
from torch.utils.data import Dataset,DataLoader
import regex as re
from googletrans import Translator
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split

/home/jaime/Escritorio/AIR/entornos/NLP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-30 13:27:30.849246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730291250.878058  290658 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730291250.884332  290658 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 13:27:30.911207: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performan

# Translate all phrases

## Translate with Google Translate

In [42]:
#tests:
#splits = {'uk': 'data/uk-00000-of-00001-86a756a5ad8b3340.parquet', 'hi': 'data/hi-00000-of-00001-35f39f96bd50e4e4.parquet', 'zh': 'data/zh-00000-of-00001-21350981874ac4f4.parquet', 'ar': 'data/ar-00000-of-00001-2338057ce0eeaf0d.parquet', 'de': 'data/de-00000-of-00001-00ccec8d5e81dbf1.parquet', 'en': 'data/en-00000-of-00001-c54cbb5edf21dd23.parquet', 'ru': 'data/ru-00000-of-00001-6818d57069197d03.parquet', 'am': 'data/am-00000-of-00001-656e19a6f1cc2383.parquet', 'es': 'data/es-00000-of-00001-723005ac428f80c9.parquet'}
#df = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox_test/" + splits["es"])
splits = {'eng_Latn': 'data/en-00000-of-00001.parquet', 
          'rus_Cyrl': 'data/ru-00000-of-00001.parquet', 
          'ukr_Cyrl': 'data/uk-00000-of-00001.parquet', 
          'deu_Latn': 'data/de-00000-of-00001.parquet', 
          'spa_Latn': 'data/es-00000-of-00001.parquet', 
          'amh_Ethi': 'data/am-00000-of-00001.parquet', 
          'zho_Hans': 'data/zh-00000-of-00001.parquet', 
          'arb_Arab': 'data/ar-00000-of-00001.parquet', 
          'hin_Deva': 'data/hi-00000-of-00001.parquet'}
translator = Translator()
all_data=[]
for key, value in splits.items():
    x=0
    final_dataframe = pd.DataFrame(columns=['toxic_sentence_not_translated','neutral_sentence_not_translated','toxic_sentence_translated','neutral_sentence_translated','original_language'])
    df = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + value)
    array = np.array(df)
    for instance in array:
        input=[]
        input.append(instance[0])
        input.append(instance[1])
        toxic_translated = translator.translate(instance[0]).text
        input.append(toxic_translated)
        non_toxic_translated = translator.translate(instance[1]).text
        input.append(non_toxic_translated)
        input.append(key)
        final_dataframe.loc[x]=input
        x+=1
    all_data.append(final_dataframe)
big_dataframe=pd.concat(all_data)
big_dataframe.to_csv('Dataset_tab.csv',sep='\t',index=False)



## Translate with NLLB-200-distilled-600M

In [112]:
#con emojis
class Translator_emojis():
    def __init__(self):
        # Use the original NLLB tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
        self.model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

    def translate(self, src_lan, tgt_lan, text):
        # Split text into words and emojis to handle emojis separately
        words = re.split(r'(\p{So}|\p{Cn})', text, flags=re.UNICODE)
        translated_text = []
        for word in words:
            if word and not re.search(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF]', word):
                # Only translate text segments, skip emojis
                text_with_lang_code = f"{src_lan} {word}"
                inputs = self.tokenizer(text_with_lang_code, return_tensors="pt")
                translated_tokens = self.model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(tgt_lan)
                )
                translated_word = self.tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
            else:
                # If it's an emoji, keep it as is
                print(u'word')
                translated_text += u'\U0001f604'

            # Append translated word or emoji to result
            translated_text.append(translated_word)
        
        return translated_text
    
#Omitiendo emojis
class Translator():
    def __init__(self):
        self.tokenizer=AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
        self.model=AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
    def translate(self,src_lan,tgt_lan,text):
        text=f"{src_lan} {text}"
        inputs = self.tokenizer(text, return_tensors="pt")
        translated_tokens = self.model.generate(** inputs, forced_bos_token_id=self.tokenizer.convert_tokens_to_ids(tgt_lan))
        translated=self.tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
        return translated

In [114]:
translator=Translator()
translator_emojis = Translator_emojis()
print(translator.translate(src_lan='spa_Latn',tgt_lan='eng_Latn',text="Example text with emoji 🤯 and symbol ♡."))
print(translator_emojis.translate(src_lan='spa_Latn',tgt_lan='eng_Latn',text="Example text with emoji 🤯 and symbol ♡."))

Example text with emoji  and symbol ♡.
['Example text with emoji ', '', 'and symbol', '♪ ♪ ♪', "- I 'm not ."]


# Mergin Datasets

In [16]:
languages = pd.read_csv('Dataset_tab.csv',sep='\t')

english = pd.read_csv('train.csv',sep='\t')
english['toxic_sentence_translated']=np.array(english['en_toxic_comment'])
english['neutral_sentence_translated']=np.array(english['en_neutral_comment'])
lang = ["eng_Latn" for i in range(len(english))]
english['original_language']=lang
english = english.rename(columns={'en_toxic_comment': 'toxic_sentence_not_translated', 'en_neutral_comment': 'neutral_sentence_not_translated'})
english.to_csv('english_lines_translated.csv',sep='\t',index=False)

In [24]:
total_dataset = pd.concat([languages,english])
total_dataset.to_csv('total_dataset_translated.csv',sep='\t',index=False)

# Loading and preparing Data

In [2]:
all_languages = pd.read_csv('total_dataset_translated.csv',sep='\t')

In [3]:
all_languages

,toxic_sentence_not_translated,neutral_sentence_not_translated,toxic_sentence_translated,neutral_sentence_translated,original_language
0,"then all of a sudden i see her , shes now got ...","All of a sudden i see her, she is all grown up.","then all of a sudden i see her , shes now got ...","All of a sudden i see her, she is all grown up.",eng_Latn
1,My page should be protected first so that wort...,My page should be protected first so that unpl...,My page should be protected first so that wort...,My page should be protected first so that unpl...,eng_Latn
2,You made a mistake you ass.,You made a mistake.,You made a mistake you ass.,You made a mistake.,eng_Latn
3,"you know more than these idiots , stay the cou...","you know more than these people , stay the cou...","you know more than these idiots , stay the cou...","you know more than these people , stay the cou...",eng_Latn
4,"piss me off , fuckin jerk , get on my nerves .",get on my nerves,"piss me off , fuckin jerk , get on my nerves .",get on my nerves,eng_Latn
...,...,...,...,...,...
23339,when they do shit like this .,when they do stuff like this,when they do shit like this .,when they do stuff like this,eng_Latn
23340,"but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more...","but if saying "" fuck that group "" is much more...","but if saying"" that group is bad"" is much more...",eng_Latn
23341,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...,"it hurts how judgemental assholes view them , ...",It hurts how judgemental that people view them...,eng_Latn
23342,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.,shit we probably literally blow that up in a w...,We probably litteralky blow that up in a week.,eng_Latn


In [4]:
array=np.array(all_languages)
X_train,X_test,y_train,y_test = train_test_split(array[:,2],array[:,3],train_size=0.8,stratify=array[:,-1]) #Estratificado por idioma para manter la distribución de clases en ambos conjuntos

In [5]:
class dataset(Dataset):
    def __init__(self,X,y):
        super().__init__()
        self.X=X
        self.y=y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, i):
        return self.X[i], self.y[i]
train_dataset = dataset(X_train,y_train)
test_dataset = dataset(X_test,y_test)
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=True)


In [91]:
a=np.array(next(iter(train_dataloader)))
b=a[0].tolist()

In [81]:
tokenizer = AutoTokenizer.from_pretrained('s-nlp/bart-base-detox')
model = BartForConditionalGeneration.from_pretrained('s-nlp/bart-base-detox')

In [92]:
inputs = tokenizer(b, return_tensors="pt", padding='longest')


In [93]:
inputs

{'input_ids': tensor([[    0,  1594,   939,   205, 13512,   118,  2463,  2156,    47,   342,
         42167, 22396, 17487,     2, 50265, 50265, 50265, 50265, 50265, 50265,
         50265, 50265, 50265],
        [    0,  7877, 27726,    82,   939,    33,   655,  1145,   479,     2,
         50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265,
         50265, 50265, 50265],
        [    0,   118,   224,  8382,  2662,    15,    84,    53,  1872,     8,
           109,  1085,   479,     2, 50265, 50265, 50265, 50265, 50265, 50265,
         50265, 50265, 50265],
        [    0,  1075, 41460,   127, 23523,  4709,  1090, 11616,  1149, 27398,
            95,  1224,   160,     5, 23523,  2888,     2, 50265, 50265, 50265,
         50265, 50265, 50265],
        [    0, 10219,    45,    28, 12103,    59,    42,   479,     2, 50265,
         50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265,
         50265, 50265, 50265],
        [    0,  3226,  1009,    14,   12

In [99]:
inputs['input_ids'][0]

tensor([    0,  1594,   939,   205, 13512,   118,  2463,  2156,    47,   342,
        42167, 22396, 17487,     2, 50265, 50265, 50265, 50265, 50265, 50265,
        50265, 50265, 50265])

In [102]:
c=model.generate(inputs['input_ids'])

/home/jaime/Escritorio/AIR/entornos/NLP/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [114]:
a[0,3]

np.str_('omfg my fucking arsehole stepdad just turned off the fucking internet')

In [116]:
tokenizer.decode(c[3])

'</s><s>omfg my stepdad just turned off the internet</s><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
class Detox():
    def __init__(self):
        self.base_model_name='facebook/bart-base'
        self.model_name = 's-nlp/bart-base-detox'
        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_name)
        self.model = BartForConditionalGeneration.from_pretrained(self.model_name)

    def detox(self,input):
        input_ids = self.tokenizer.encode(input, return_tensors='pt')
        output_ids = self.model.generate(input_ids)
        output_text = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return output_text
    
    def train(self, dataloader):